In [ ]:
#imports
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [ ]:
#training data
df_train=pd.read_csv('/content/drive/My Drive/minor/hindi_dataset/hindi_dataset.tsv',sep='\t', encoding="utf-8")
df_train=df_train[df_train.task_2!='NONE']
df_train.head()

,text_id,text,task_1,task_2,task_3
1,hasoc_hi_5648,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,HOF,PRFN,UNT
2,hasoc_hi_164,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,HOF,PRFN,TIN
18,hasoc_hi_6865,"#नीच'समानार्थी शब्द #मोदी' दिला तर चालेल की, ल...",HOF,OFFN,TIN
20,hasoc_hi_3763,इस मादरचोद को धुण्डके गांड मे गोली मरो @Uppolice,HOF,PRFN,UNT
22,hasoc_hi_5898,You cry in front of ur god Of being deceived ...,HOF,PRFN,UNT


In [ ]:
#training and validation split
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_train['text'], df_train['task_2'],test_size=0.15,random_state=140) 

In [ ]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder() #label encoder
train_y = encoder.fit_transform(train_y) #encoding label for training data
valid_y = encoder.fit_transform(valid_y) #encoding label for validation data

In [ ]:
#testing data
df_test=pd.read_csv('/content/drive/My Drive/minor/hindi_dataset/hasoc2019_hi_test_gold_2919.tsv', sep='\t',encoding="utf-8") 
df_test=df_test[df_test.task_2!='NONE']
test_y = encoder.fit_transform(df_test['task_2']) #encoding the label of testing data

In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df_train['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)  #get count vector features for training data
xvalid_count =  count_vect.transform(valid_x)  #get count vector features for validation data
x_count =  count_vect.transform(df_test['text'])  #get count vector features for testing data

In [ ]:
def model(classifier, feature_vector_train, label, feature_vector_valid,feature_vector_test):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions_valid = classifier.predict(feature_vector_valid)

    # predict the labels on testing dataset
    predictions_test = classifier.predict(feature_vector_test)

    print("classification report for validation")
    print(metrics.classification_report(predictions_valid,valid_y))
    print("classification report for testing")
    print(metrics.classification_report(predictions_test,test_y))


In [ ]:
# calling the xgboost classifier model for training ,validation and testing
model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc(),x_count.tocsc())

classification report for validation
              precision    recall  f1-score   support

           0       0.30      0.57      0.39        51
           1       0.53      0.39      0.45       121
           2       0.78      0.72      0.75       199

    accuracy                           0.59       371
   macro avg       0.54      0.56      0.53       371
weighted avg       0.63      0.59      0.60       371

classification report for testing
              precision    recall  f1-score   support

           0       0.32      0.47      0.38       129
           1       0.45      0.48      0.46       183
           2       0.85      0.63      0.73       293

    accuracy                           0.55       605
   macro avg       0.54      0.53      0.52       605
weighted avg       0.62      0.55      0.57       605

